# Project 4: Lid Driven Cavity


- Resource: https://github.com/davidlin001/flow_sims/tree/master/SIMPLE


`Libraries`


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from solvers import *

`Initial Variables`


In [18]:
Nx = Ny = 100
l = 1.0
nu = 0.01
velocity = 1.0
Re = 1.0

dx = l / Nx
dy = l / Ny

x = np.linspace(0.0, l, Nx)
y = np.linspace(0.0, l, Ny)
X,Y = np.meshgrid(x,y)

u = u_star = np.zeros((Nx+2, Ny+2))
v = v_star =  np.zeros((Nx+2,Ny + 2))
p = p_star = p_prime = np.zeros((Nx+2, Ny+2))

`Coefficients`


In [19]:
A_p = A_w = A_s = A_e = A_n = np.zeros((Nx+2, Ny+2))
Ap_p = Ap_w = Ap_s = Ap_e = Ap_n = np.zeros((Nx+2, Ny+2))

source_x = np.zeros(Nx*Ny) 
source_y = np.zeros(Nx*Ny) 
source_p = np.zeros(Nx*Ny) 
u_face = np.zeros((Ny+2,Nx+1))
v_face = np.zeros((Ny+1,Nx+2))

A_momentum = np.zeros((Nx*Ny, Nx*Ny))
A_pressure = np.zeros((Nx*Ny, Nx*Ny))

In [20]:
def tdma_solver(a, b, c, d):
    n = len(b)
    # Forward sweep
    for i in range(1, n):
        w = a[i-1] / b[i-1]
        b[i] -= w * c[i-1]
        d[i] -= w * d[i-1]
    
    # Backward substitution
    x = np.zeros(n)
    x[-1] = d[-1] / b[-1]
    for i in range(n-2, -1, -1):
        x[i] = (d[i] - c[i] * x[i+1]) / b[i]
    
    return x

def ADI_solver(phi, D_x, D_y, S, dx, dy, dt, max_iters=1000, tol=1e-6):
    n_y, n_x = phi.shape
    
    # Calculate constants for the 1D implicit step
    alpha_x = D_x * dt / (dx ** 2)
    alpha_y = D_y * dt / (dy ** 2)

    for _ in range(max_iters):
        # Step 1: Solve for phi in the x-direction (using TDMA)
        for i in range(1, n_y-1):
            # Prepare the tridiagonal system for row i (x-direction)
            a = np.full(n_x-2, -alpha_x)  # Lower diagonal
            b = np.full(n_x-1, 1 + 2 * alpha_x)  # Main diagonal
            c = np.full(n_x-2, -alpha_x)  # Upper diagonal
            d = phi[i, 1:-1] + alpha_x * (phi[i, 2:] + phi[i, :-2])  # Right-hand side
            
            # Solve the tridiagonal system for this row
            phi[i, 1:-1] = tdma_solver(a, b, c, d)
        
        # Step 2: Solve for phi in the y-direction (using TDMA)
        for j in range(1, n_x-1):
            # Prepare the tridiagonal system for column j (y-direction)
            a = np.full(n_y-2, -alpha_y)  # Lower diagonal
            b = np.full(n_y-1, 1 + 2 * alpha_y)  # Main diagonal
            c = np.full(n_y-2, -alpha_y)  # Upper diagonal
            d = phi[1:-1, j] + alpha_y * (phi[2:, j] + phi[:-2, j])  # Right-hand side
            
            # Solve the tridiagonal system for this column
            phi[1:-1, j] = tdma_solver(a, b, c, d)
        
        # Check convergence (L2 norm)
        residual = np.linalg.norm(S - (D_x * (phi[2:, 1:-1] - 2 * phi[1:-1, 1:-1] + phi[:-2, 1:-1]) / (dx ** 2) +
                                       D_y * (phi[1:-1, 2:] - 2 * phi[1:-1, 1:-1] + phi[1:-1, :-2]) / (dy ** 2)))
        if residual < tol:
            print(f"Converged after {_+1} iterations.")
            break

    return phi, residual


In [21]:
def mesh_linking(A_p,A_e,A_s,A_n,A_w, u_star,v_star, u_face,v_face,p, source_x, source_y, alpha_uv, dy, dx, Re, velocity):
    D_e = dy / (dx * Re)
    D_w = dy / (dx * Re)
    D_n = dx / (dy * Re)
    D_s = dx / (dy * Re)

    ## For internal nodes
    for i in range(2, Ny):
        for j in range(2, Nx):
            n = (i-1) * Nx + (j - 1)
            
            Fe = dy * u_face[i,j]
            Fw = dy * u_face[i, j-1]
            Fn = dx * v_face[i-1, j]
            Fs = dx * v_face[i,j]

            A_e[i,j] = D_e + max(0.0, -Fe)
            A_w[i,j] = D_w + max(0.0, Fw)
            A_n[i,j] = D_n + max(0.0, -Fn)
            A_s[i,j] = D_s + max(0.0, Fs)
            A_p[i,j] = D_e + D_w + D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
            source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
            source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]
    ## Left wall
    j = 1
    for i in range(2, Ny):
        n = (i - 1)* Nx
        Fe = dy * u_face[i,j]
        Fw = dy * u_face[i, j-1]
        Fn = dx * v_face[i-1, j]
        Fs = dx * v_face[i,j]

        A_e[i,j] = D_e + max(0.0, -Fe)
        A_n[i,j] = D_n + max(0.0, -Fn)
        A_s[i,j] = D_s + max(0.0, Fs)
        A_p[i,j] = D_e + 2 * D_w + D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
        source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
        source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]

    ## for bottom wall
    i = Ny
    for j in range(2, Nx):
        n = (Ny - 1)*Nx + (j - 1)
        Fe = dy * u_face[i,j]
        Fw = dy * u_face[i, j-1]
        Fn = dx * v_face[i-1, j]
        Fs = dx * v_face[i,j]

        A_e[i,j] = D_e + max(0.0, -Fe)
        A_w[i,j] = D_w + max(0.0, Fw)
        A_n[i,j] = D_n + max(0.0, -Fn)

        A_p[i,j] = D_e + D_w + D_n + 2 * D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
        source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
        source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]
    
    ## right wall
    j = Nx
    for i in range(2, Ny):
        n = i*Nx - 1
        Fe = dy * u_face[i,j]
        Fw = dy * u_face[i, j-1]
        Fn = dx * v_face[i-1, j]
        Fs = dx * v_face[i,j]

        A_w[i,j] = D_w + max(0.0, Fw)
        A_n[i,j] = D_n + max(0.0, -Fn)
        A_s[i,j] = D_s + max(0.0, Fs)
        A_p[i,j] = 2* D_e + D_w + D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
        source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
        source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]

    ## top wall
    i = 1
    for j in range(2, Nx):
        n = j - 1
        Fe = dy * u_face[i,j]
        Fw = dy * u_face[i, j-1]
        Fn = dx * v_face[i-1, j]
        Fs = dx * v_face[i,j]

        A_e[i,j] = D_e + max(0.0, -Fe)
        A_w[i,j] = D_w + max(0.0, Fw)
        A_s[i,j] = D_s + max(0.0, Fs)
        A_p[i,j] = D_e + D_w + 2*D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
        source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
        source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]
    
    ## top left corner
    i = 1
    j = 1
    n = 0
    Fe = dy * u_face[i,j]
    Fw = dy * u_face[i, j-1]
    Fn = dx * v_face[i-1, j]
    Fs = dx * v_face[i,j]

    A_e[i,j] = D_e + max(0.0, -Fe)
    A_s[i,j] = D_s + max(0.0, Fs)
    A_p[i,j] = D_e + 2 * D_w + 2 * D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
    source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j] + velocity  * (2 * D_n + max(0.0, -Fn))
    source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]

    ## top right corner
    i = 1
    j = Nx
    n = Nx - 1

    Fe = dy * u_face[i,j]
    Fw = dy * u_face[i, j-1]
    Fn = dx * v_face[i-1, j]
    Fs = dx * v_face[i,j]

    A_w[i,j] = D_w + max(0.0, Fw)
    A_s[i,j] = D_s + max(0.0, Fs)
    A_p[i,j] = 2 * D_e + D_w + 2 * D_n + D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
    source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j] + velocity * (2 * D_n + max(0.0, Fn))
    source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]

    ## Bottom left corner

    i = Ny
    j = 1
    n = (Ny - 1) * Nx
    Fe = dy * u_face[i,j]
    Fw = dy * u_face[i, j-1]
    Fn = dx * v_face[i-1, j]
    Fs = dx * v_face[i,j]

    A_e[i,j] = D_e + max(0.0, -Fe)
    A_n[i,j] = D_n + max(0.0, -Fn)
    A_p[i,j] = D_e + 2*D_w + D_n + 2 *D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
    source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
    source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]

    ## Bottom right corner
    i = Ny
    j = Nx
    n = Nx * Ny - 1
    Fe = dy * u_face[i,j]
    Fw = dy * u_face[i, j-1]
    Fn = dx * v_face[i-1, j]
    Fs = dx * v_face[i,j]

    A_w[i,j] = D_w + max(0.0, Fw)
    A_n[i,j] = D_n + max(0.0, -Fn)
    A_p[i,j] = 2 * D_e + D_w + D_n + 2*D_s + max(0.0, Fe) + max(0.0, -Fw) +  max(0.0, Fn) + max(0.0, -Fs)
    source_x[n] = 0.5 * alpha_uv*(p[i, j-1] - p[i, j+1]) * dy + (1-alpha_uv) * A_p[i,j]*u_star[i,j]
    source_y[n] = 0.5 * alpha_uv*(p[i+1, j] - p[i-1, j]) * dy + (1-alpha_uv) * A_p[i,j]*v_star[i,j]
    A_e *= alpha_uv
    A_w *= alpha_uv
    A_n *= alpha_uv
    A_s *= alpha_uv

In [22]:

def build_coefficient_matrix(Nx, Ny, A, ap, ae, aw, an, as_):
    # Initialize A with zeros
    A.fill(0)

    # Top wall
    i = 1
    for j in range(1, Nx - 1):
        n = j - 1
        A[n, n] = ap[i, j]
        A[n, n - 1] = -aw[i, j]
        A[n, n + 1] = -ae[i, j]
        A[n, n + Nx] = -as_[i, j]

    # Interior cells
    for i in range(1, Ny - 1):
        for j in range(1, Nx - 1):
            n = (i - 1) * Nx + (j - 1)
            A[n, n] = ap[i, j]
            A[n, n - 1] = -aw[i, j]
            A[n, n + 1] = -ae[i, j]
            A[n, n - Nx] = -an[i, j]
            A[n, n + Nx] = -as_[i, j]

    # Left wall
    j = 1
    for i in range(1, Ny - 1):
        n = (i - 1) * Nx
        A[n, n] = ap[i, j]
        A[n, n + 1] = -ae[i, j]
        A[n, n - Nx] = -an[i, j]
        A[n, n + Nx] = -as_[i, j]

    # Right wall
    j = Nx - 1
    for i in range(1, Ny - 1):
        n = i * Nx - 1
        A[n, n] = ap[i, j]
        A[n, n - 1] = -aw[i, j]
        A[n, n - Nx] = -an[i, j]
        A[n, n + Nx] = -as_[i, j]

    # Bottom wall
    i = Ny - 1
    for j in range(1, Nx - 1):
        n = (Ny - 1) * Nx + (j - 1)
        A[n, n] = ap[i, j]
        A[n, n - 1] = -aw[i, j]
        A[n, n + 1] = -ae[i, j]
        A[n, n - Nx] = -an[i, j]

    # Top left corner
    n = 0
    i = 1
    j = 1
    A[n, n] = ap[i, j]
    A[n, n + 1] = -ae[i, j]
    A[n, n + Nx] = -as_[i, j]

    # Top right corner
    i = 1
    j = Nx - 1
    n = Nx - 1
    A[n, n] = ap[i, j]
    A[n, n - 1] = -aw[i, j]
    A[n, n + Nx] = -as_[i, j]

    # Bottom left corner
    n = (Ny - 1) * Nx
    j = 1
    i = Ny - 1
    A[n, n] = ap[i, j]
    A[n, n + 1] = -ae[i, j]
    A[n, n - Nx] = -an[i, j]

    # Bottom right corner
    n = (Nx * Ny) - 1
    i = Ny - 1
    j = Nx - 1
    A[n, n] = ap[i, j]
    A[n, n - 1] = -aw[i, j]
    A[n, n - Nx] = -an[i, j]

    return A

def face_velocity(u, v, u_face, v_face, p, A_p, alpha_uv, dy):
    # u face velocity
    for i in range(1, Ny - 1):
        for j in range(1, Nx - 1):
            u_face[i, j] = (
                0.5 * (u[i, j] + u[i, j + 1])
                + 0.25 * alpha_uv * (p[i, j + 1] - p[i, j - 1]) * dy / A_p[i, j]
                + 0.25 * alpha_uv * (p[i, j + 2] - p[i, j]) * dy / A_p[i, j + 1]
                - 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (p[i, j + 1] - p[i, j]) * dy
            )

    # v face velocity
    for i in range(2, Ny):
        for j in range(1, Nx):
            v_face[i - 1, j] = (
                0.5 * (v[i, j] + v[i - 1, j])
                + 0.25 * alpha_uv * (p[i - 1, j] - p[i + 1, j]) * dy / A_p[i, j]
                + 0.25 * alpha_uv * (p[i - 2, j] - p[i, j]) * dy / A_p[i - 1, j]
                - 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (p[i - 1, j] - p[i, j]) * dy
            )




In [23]:
def pressure_correction_link_coefficients(u, u_face, v_face, Ap_p, Ap_e, Ap_w, Ap_n, Ap_s, source_p, A_p, alpha_uv, dx, dy, Nx, Ny):

    # Interior cells
    for i in range(1, Ny - 1):
        for j in range(1, Nx - 1):
            n = (i - 1) * (Nx - 2) + (j - 1)

            Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
            Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
            Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
            Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
            Ap_p[i, j] = Ap_e[i, j] + Ap_w[i, j] + Ap_n[i, j] + Ap_s[i, j]

            source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Top boundary
    i = 0
    for j in range(1, Nx - 1):
        n = j - 1
        Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
        Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
        Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
        Ap_p[i, j] = Ap_e[i, j] + Ap_w[i, j] + Ap_s[i, j]

        source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Left boundary
    j = 0
    for i in range(1, Ny - 1):
        n = (i - 1) * (Nx - 2)
        Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
        Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
        Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
        Ap_p[i, j] = Ap_e[i, j] + Ap_n[i, j] + Ap_s[i, j]

        source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Right boundary
    j = Nx - 1
    for i in range(1, Ny - 1):
        n = i * (Nx - 2) - 1
        Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
        Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
        Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
        Ap_p[i, j] = Ap_w[i, j] + Ap_n[i, j] + Ap_s[i, j]

        source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Bottom boundary
    i = Ny - 1
    for j in range(1, Nx - 1):
        n = (Ny - 2) * (Nx - 2) + (j - 1)
        Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
        Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
        Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
        Ap_p[i, j] = Ap_e[i, j] + Ap_w[i, j] + Ap_n[i, j]

        source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Top-left corner
    i, j, n = 0, 0, 0
    Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
    Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
    Ap_p[i, j] = Ap_e[i, j] + Ap_s[i, j]

    source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Top-right corner
    i, j, n = 0, Nx - 1, Nx - 2
    Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
    Ap_s[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i + 1, j]) * (dx * dx)
    Ap_p[i, j] = Ap_w[i, j] + Ap_s[i, j]

    source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Bottom-left corner
    i, j, n = Ny - 1, 0, (Ny - 2) * (Nx - 2)
    Ap_e[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j + 1]) * (dy * dy)
    Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
    Ap_p[i, j] = Ap_e[i, j] + Ap_n[i, j]

    source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx

    # Bottom-right corner
    i, j, n = Ny - 1, Nx - 1, (Ny - 1) * (Nx - 2) - 1
    Ap_w[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j - 1]) * (dy * dy)
    Ap_n[i, j] = 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i - 1, j]) * (dx * dx)
    Ap_p[i, j] = Ap_w[i, j] + Ap_n[i, j]

    source_p[n] = -(u_face[i, j] - u_face[i, j - 1]) * dy - (v_face[i - 1, j] - v_face[i, j]) * dx


In [24]:
def correct_pressure(p_star, p, p_prime, alpha_p):
    n_y, n_x = p.shape[0] - 2, p.shape[1] - 2  # Interior grid size excluding boundry cells

    # Top wall
    p_prime[0, 1:n_x+1] = p_prime[1, 1:n_x+1]

    # Left wall
    p_prime[1:n_y+1, 0] = p_prime[1:n_y+1, 1]

    # Right wall
    p_prime[1:n_y+1, n_x+1] = p_prime[1:n_y+1, n_x]

    # Bottom wall
    p_prime[n_y+1, 1:n_x+1] = p_prime[n_y, 1:n_x+1]

    # Top-left corner
    p_prime[0, 0] = (p_prime[1, 1] + p_prime[0, 1] + p_prime[1, 0]) / 3

    # Top-right corner
    p_prime[0, n_x+1] = (p_prime[0, n_x] + p_prime[1, n_x] + p_prime[1, n_x+1]) / 3

    # Bottom-left corner
    p_prime[n_y+1, 0] = (p_prime[n_y, 0] + p_prime[n_y, 1] + p_prime[n_y+1, 1]) / 3

    # Bottom-right corner
    p_prime[n_y+1, n_x+1] = (p_prime[n_y, n_x+1] + p_prime[n_y+1, n_x] + p_prime[n_y, n_x]) / 3

    # Correct the pressure field
    p_star[:, :] = p + alpha_p * p_prime


In [25]:
def correct_cell_center_velocity(u, v, u_star, v_star, p_prime, A_p, alpha_uv, dx, dy):
    n_y, n_x = u.shape[0] - 2, u.shape[1] - 2  # Interior grid size excluding ghost cells

    # Correct u velocity (interior cells)
    for i in range(1, n_y+1): 
        for j in range(2, n_x): 
            u_star[i, j] = u[i, j] + 0.5 * alpha_uv * (p_prime[i, j-1] - p_prime[i, j+1]) * dy / A_p[i, j]

    # Left boundary (j = 1)
    j = 1
    for i in range(1, n_y+1):
        u_star[i, j] = u[i, j] + 0.5 * alpha_uv * (p_prime[i, j] - p_prime[i, j+1]) * dy / A_p[i, j]

    # Right boundary (j = n_x)
    j = n_x
    for i in range(1, n_y+1):
        u_star[i, j] = u[i, j] + 0.5 * alpha_uv * (p_prime[i, j-1] - p_prime[i, j]) * dy / A_p[i, j]

    # Correct v velocity (interior cells)
    for i in range(2, n_y): 
        for j in range(1, n_x+1): 
            v_star[i, j] = v[i, j] + 0.5 * alpha_uv * (p_prime[i+1, j] - p_prime[i-1, j]) * dx / A_p[i, j]

    # Top boundary (i = 1)
    i = 1
    for j in range(1, n_x+1): 
        v_star[i, j] = v[i, j] + 0.5 * alpha_uv * (p_prime[i+1, j] - p_prime[i, j]) * dx / A_p[i, j]

    # Bottom boundary (i = n_y)
    i = n_y
    for j in range(1, n_x+1):
        v_star[i, j] = v[i, j] + 0.5 * alpha_uv * (p_prime[i, j] - p_prime[i-1, j]) * dx / A_p[i, j]


In [26]:
def correct_face_velocity(u_face, v_face, p_prime, A_p, alpha_uv, dx, dy):

    n_y, n_x = p_prime.shape[0] - 2, p_prime.shape[1] - 2
    for i in range(1, n_y+1): 
        for j in range(1, n_x): 
            u_face[i, j] = u_face[i, j] + 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i, j+1]) * (p_prime[i, j] - p_prime[i, j+1]) * dy

    for i in range(2, n_y+1): 
        for j in range(1, n_x+1): 
            v_face[i-1, j] = v_face[i-1, j] + 0.5 * alpha_uv * (1 / A_p[i, j] + 1 / A_p[i-1, j]) * (p_prime[i, j] - p_prime[i-1, j]) * dx


`Boundry Conditions`


In [27]:
u[0, 1:Nx+1] = 1.0
u_star[0, 1:Nx+1] = 1.0
u_face[0, 1:Nx] = 1.0

In [28]:
max_outer_iterations = 1000
epsilon_uv = 1e-6
epsilon_p = 1e-6

In [ ]:
for n in range(1, max_outer_iterations + 1):
    # Link coefficients
    mesh_linking(A_p, A_e, A_w, A_n, A_s, source_x, source_y, u_face, v_face, p, u_star, v_star, 0.7, dy, dx, 1/nu, 1)

    # Build coefficient matrix for momentum equation
    build_coefficient_matrix(A_momentum, A_p, A_e, A_w, A_n, A_s)

    # Solve for u and v velocities
    u_star, l2_norm_x = ADI_solver(A_momentum, source_x, u, l2_norm_x, epsilon_uv)
    v_star, l2_norm_y = ADI_solver(A_momentum, source_y, v, l2_norm_y, epsilon_uv)

    # Update face velocities
    face_velocity(u, v, u_face, v_face, p, A_p)

    # Pressure correction link coefficients
    pressure_correction_link_coefficients(u, u_face, v_face, Ap_p, Ap_e, Ap_w, Ap_n, Ap_s, source_p, A_p)

    # Solve the pressure correction using ADI solver
    p_prime, l2_norm_p = ADI_solver(Ap_p, source_p, p_prime, Nx, Ny, dx, dy, alpha=1.0, epsilon=epsilon_p)

    # Correct pressure and velocity
    correct_pressure(p_star, p, p_prime)
    correct_cell_center_velocity(u, v, u_star, v_star, p_prime, A_p)
    correct_face_velocity(u_face, v_face, p_prime, A_p)

    # Update pressure
    p = p_star

    # Output iteration info
    print(f"{n} {l2_norm_x} {l2_norm_y} {l2_norm_p}")

    # Check convergence
    if l2_norm_x < 1e-5 and l2_norm_y < 1e-5 and l2_norm_p < 1e-6:
        print("Converged!")
        break


IndexError: index 100 is out of bounds for axis 0 with size 100